In [1]:
#push!(LOAD_PATH,"julia/src")
#import Simulation

4-element Array{String,1}:
 "@"
 "@v#.#"
 "@stdlib"
 "julia/src"

In [18]:
0x0

0x00

In [230]:
include("julia/src/Simulation.jl") # might need some time to compile

Main.Simulation

In [231]:
using Random
#loads and pre-processes data that is constant during the simulation time
params = Simulation.load_params(
    MersenneTwister(3),
    population_path="data/simulations/wroclaw-population-orig.csv.gz", 
    incubation_time_samples_path="test/models/assets/incubation_period_distribution.npy", 
    t0_to_t1_samples_path="test/models/assets/t1_distribution.npy",
    t0_to_t2_samples_path="test/models/assets/t1_t2_distribution.npy");

params.progressions[1] |> display
params.household_ptrs[1] |> display

Main.Simulation.Progression(Main.Simulation.Severe, 9.0f0, NaN32, 9.5f0)

(0x00000001, 0x00000001)

In [237]:
state = Simulation.SimState(params.progressions |> length, seed=123)

push!(state.queue, Simulation.OutsideInfectionEvent(0.0,1))

for iter  in 0:20
    #println("iteration=$iter")
    if isempty(state.queue)
        @info "Empty queue after $iter events"
        break
    end
    #state.individuals[1] |> display
    #state.individuals[491936] |> display
    event = pop!(state.queue)
    state.time = Simulation.time(event)
    event |> print
    result = Simulation.execute!(state, params, event)
    result |> println
    
end

id = 0x000781a0
state.individuals[id]
Simulation.householdof(params, id) |> collect

Main.Simulation.OutsideInfectionEvent(0.0f0, 0x00000001)true
Main.Simulation.BecomeInfectiousEvent(9.0f0, 0x00000001)true
Main.Simulation.TransmissionEvent(9.336979f0, 0x000781a0, 0x00000001, Main.Simulation.ConstantKernelContact)true
Main.Simulation.SevereSymptomsEvent(9.5f0, 0x00000001)true
Main.Simulation.GoHospitalEvent(9.5f0, 0x00000001)true
Main.Simulation.DetectedEvent(9.5f0, 0x00000001, false)true
Main.Simulation.BackTrackedEvent(11.5f0, 0x000781a0)true
Main.Simulation.QuarantinedEvent(11.5f0, 0x000781a0)true
Main.Simulation.QuarantinedEvent(11.5f0, 0x000781a0)false
Main.Simulation.QuarantinedEvent(11.5f0, 0x000781a0)false
Main.Simulation.QuarantinedEvent(11.5f0, 0x000781a0)false
Main.Simulation.BecomeInfectiousEvent(14.336979f0, 0x000781a0)true
Main.Simulation.TransmissionEvent(14.690809f0, 0x0004fe6d, 0x000781a0, Main.Simulation.ConstantKernelContact)true
Main.Simulation.TransmissionEvent(16.24155f0, 0x0008fc6b, 0x000781a0, Main.Simulation.ConstantKernelContact)true
Main.Simu

4-element Array{UInt32,1}:
 0x000781a0
 0x000781a1
 0x000781a2
 0x000781a3

In [73]:
state.individuals[491936]

Main.Simulation.IndividualState(Main.Simulation.Healthy, Main.Simulation.Free, 0, false)

In [52]:
## creates mutable simulation state
state = Simulation.SimState(params.progressions |> length, seed=123)

# add initial event (infect the person no1 at time 0)
push!(state.queue, Simulation.OutsideInfectionEvent(0.0,1))
# (the first person must be infectious for significant amount of time for the epidemics to kick off)

# the first run includes compilation time, run twice to get meaningful results (precompilation will be added soon)
@time Simulation.simulate!(state, params)


UndefVarError: UndefVarError: params_id not defined

In [279]:
using DataStructures
i1, i2 = searchequalrange(state.infections, 1)
state.infections[i1]


Main.Simulation.TransmissionEvent(18.69558f0, 491936, 1, Main.Simulation.ConstantKernelContact)

In [10]:
infections = SortedMultiDict{Int32,Int32}()
push!(infections, 0=>1)
push!(infections, 1=>2)
push!(infections, 1=>3)
push!(infections, 2=>4)
push!(infections, 3=>5)
push!(infections, 3=>6)
for (k,v) in inclusive(infections, searchequalrange(infections, 3)...)
    println("$k ---> $v")
end

3 ---> 5
3 ---> 6


In [8]:
inclusive(infections, i1, i2) |> sizeof

24